In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# If you don't have csv
# Download it using 
# df = yf.Ticker('Reliance.NS').history(period='max', interval='1mo', actions=False)
df = pd.read_csv("reliance_monthly.csv")
df.reset_index()
df.set_index('Date')

,Open,High,Low,Close,Volume
Date,,,,,
1996-01-01,10.471230,11.081626,7.766588,9.028243,3.587400e+09
1996-02-01,8.938852,12.511840,8.913313,10.821119,6.910292e+09
1996-03-01,10.905402,12.156842,9.832740,10.565725,5.434432e+09
1996-04-01,10.611698,13.408283,10.522310,12.496519,4.130763e+09
1996-05-01,13.638136,13.786266,11.520906,12.174718,5.496793e+09
...,...,...,...,...,...
2022-02-01,2408.000000,2456.399902,2243.000000,2359.550049,1.158835e+08
2022-03-01,2359.550049,2688.000000,2180.000000,2634.750000,1.450252e+08
2022-04-01,2636.000000,2856.149902,2521.800049,2790.250000,1.362090e+08


# Methodology
I will be identifying demand zone structure and supply zone structure on basis of candlestick structure. After identifying demand zone, we can further optimize it by checking its strength, identifying factors such as consolidation and can use it to automate and make our chart analysis process easier.

Boring Candles -> Candles which match the criterion -> Size of body less than 50% of size of wicks
Exciting Candles -> Candles which match the criterion -> Size of body greater than 50% of size of wicks.

Demand zones -> Structure which satisfies this order of candlesticks in given order
-> Red Exciting Candle followed by (1-4) boring candles followed with Strong Green Exciting Candle

![Drop Base Rally Demand Zone](./images/dbr_zone.png)

In [3]:
df['boringCandle'] = (abs(df['Open']-df['Close']) / abs(df['High']-df['Low']) < 0.5)
df['greenExcitingCandle'] = (abs(df['Open']-df['Close'])/abs(df['High']-df['Low']) > 0.5) & (df['Open']<df['Close'])
df['redExcitingCandle'] = (abs(df['Open']-df['Close'])/abs(df['High']-df['Low']) > 0.5) & (df['Open']>df['Close'])

In [4]:
df['demandZones'] = (df['redExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']) \
	| (df['redExcitingCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']) \
	| (df['redExcitingCandle'].shift(4) & df['boringCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']) \
	| (df['redExcitingCandle'].shift(5) & df['boringCandle'].shift(4) & df['boringCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle'])
df['supplyZones'] = (df['greenExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle']) \
	| (df['greenExcitingCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle']) \
	| (df['greenExcitingCandle'].shift(4) & df['boringCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle']) \
	| (df['greenExcitingCandle'].shift(5) & df['boringCandle'].shift(4) & df['boringCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle'])


In [5]:
df.iloc[np.where(df['demandZones'] == True)]

,Date,Open,High,Low,Close,Volume,boringCandle,greenExcitingCandle,redExcitingCandle,demandZones,supplyZones
29,1998-03-01,17.736742,20.395047,17.494075,19.303045,1.166477e+09,False,True,False,True,False
39,1998-12-01,12.685675,14.145039,12.628891,13.582870,9.609394e+08,False,True,False,True,False
53,2000-01-01,28.093851,40.443315,28.093851,38.816830,1.742788e+09,False,True,False,True,False
77,2001-11-01,31.658862,38.310919,31.024452,35.717850,3.294383e+08,False,True,False,True,False
96,2003-05-01,34.914729,38.419561,32.713874,37.987022,4.233989e+08,False,True,False,True,False
171,2009-03-01,274.967724,354.345162,249.590239,341.220032,4.532050e+08,False,True,False,True,False
177,2009-09-01,453.124944,501.261663,430.410521,492.701782,3.314368e+08,False,True,False,True,False
191,2010-10-01,449.716492,503.211017,448.809814,496.977509,2.218708e+08,False,True,False,True,False
196,2011-03-01,441.420379,478.277176,437.022963,475.602448,1.992958e+08,False,True,False,True,False
204,2011-10-01,361.738966,413.311474,348.274340,401.218445,1.477017e+08,False,True,False,True,False
